# Web scrapping

### Instalar librarías con conda:

  conda install requests beautifulsoup4

- Distinguir URL base de URL con parámetros
- Como funcionan os parámetros? (web dinámica vs estática)
- Vendo o HTML e a árbore DOM con Developer tools. Diferenciar tags con class e id.

### Visitar a web de exemplo

Imos empregar a web de exemplo: <https://realpython.github.io/fake-jobs/>

## Obtendo unha URL con requests

Este é o modo máis simple de descargar unha web.

In [ ]:
import requests

URL = "https://realpython.github.io/fake-jobs/"
page = requests.get(URL)

# Dentro de .text teremos o código da páxina
page.text

## Engadindo o parser

In [ ]:
from bs4 import BeautifulSoup

#Para poder parsear HTML
soup = BeautifulSoup(page.content, "html.parser")

## Atopar elementos por ID

In [ ]:
results = soup.find(id="ResultsContainer")

print(results.prettify())

## Atopar elementos por class

In [ ]:
job_elements = results.find_all("div", class_="card-content")

# Iterando polos elementos

In [ ]:
for job_element in job_elements:
    print(job_element.prettify(), end="\n"*2)

O anterior xera demasiado HTML, mellor collemos tan so partes: Traballo, compañía e ubicación

In [ ]:
for job_element in job_elements:
    title_element = job_element.find("h2", class_="title")
    company_element = job_element.find("h3", class_="company")
    location_element = job_element.find("p", class_="location")
    print(title_element)
    print(company_element)
    print(location_element)
    print()

Tendo en conda que cada job_element é outro obxecto tipo BeautifulSoup, podemos quitar o HTML molesto

Tamén metemos o método .strip() para quitar espacios ao principio e final: <https://www.w3schools.com/python/ref_string_strip.asp>

In [ ]:
for job_element in job_elements:
    title_element = job_element.find("h2", class_="title")
    company_element = job_element.find("h3", class_="company")
    location_element = job_element.find("p", class_="location")
    print(title_element.text.strip())
    print(company_element.text.strip())
    print(location_element.text.strip())
    print()

****Tamén podemos buscar os elementos por clase que conteñen algún texto****

In [ ]:
python_jobs = results.find_all("h2", string="Senior Python Developer")
for i in python_jobs:
    print(i)

print ("---")
python_jobs = results.find_all("h2", string="Python")
for i in python_jobs:
    print(i)

Non amosa resultados porque busca un texto que sexa exactamente igual. Espazos en branco, letras maiúsculas ou minúsculas, guións e outras variacións farán que non se atopen os resultados como queremos.

In [ ]:
python_jobs = results.find_all(
    "h2", string=lambda text: "python" in text.lower()
)

#### Imprimimos resultados

In [ ]:
print (python_jobs)

#### Mellor elemento a elemento

In [ ]:
for job in python_jobs:
    print (job)
    print()

#### E aínda mellor se quitamos o HTML

In [ ]:
for job in python_jobs:
    print (job.text.strip())

#### E se xuntamos todo...

In [ ]:

python_jobs = results.find_all(
    "h2", string=lambda text: "python" in text.lower()
)

for job_element in python_jobs:
    title_element = job_element.find("h2", class_="title")
    company_element = job_element.find("h3", class_="company")
    location_element = job_element.find("p", class_="location")
    print(title_element.text.strip())
    print(company_element.text.strip())
    print(location_element.text.strip())
    print()

#### Vaia! Fallou! Por que?
**Pista**:

In [ ]:
for job_element in python_jobs:
    title_element = job_element.find("h2", class_="title")
    print (title_element)

Hai elementos h2 coa clase title que inclúan a información que buscamos?

So temos o nome do traballo. De ahí que non temos nada mais: 

    <h2 class="title is-5">Senior Python Developer</h2>

Teríamos que acceder ao pai e de ahí sacar un obxecto que nos permita acceder ás súas propiedades

In [ ]:
python_jobs = results.find_all(
    "h2", string=lambda text: "python" in text.lower()
)

python_job_elements = [
    h2_element.parent.parent.parent for h2_element in python_jobs
]

for job_element in python_job_elements:
    links = job_element.find_all("a")
    for link in links:
        link_url = link["href"]
        print(f"Apply here: {link_url}\n")

Duas ligazóns? Non hai problema se queremos só a segunda...

In [ ]:
for job_element in python_job_elements:
    link_url = job_element.find_all("a")[1]["href"]
    print(f"Apply here: {link_url}\n")

**Exercicio**: Meter na BD MySQL os resultados nunha táboa.

    CREATE TABLE fakejob(
	    position VARCHAR(200),
	    company VARCHAR(200),
        address VARCHAR(200),
	    pubDate VARCHAR(200),
	    url VARCHAR(250)
    );

Scrapping de dous ditios mais

In [47]:
import pymysql
from sqlalchemy.engine import create_engine

db_host = "localhost"
db_port=3306
db_user = "usuario"
db_passwd="abc123."
db_name="employees"

#Xerar a cadea de conexión en base aos parámetros anteriores
connectionString=f'mysql+pymysql://{db_user}:{db_passwd}@{db_host}:{db_port}/{db_name}'

engine = create_engine(connectionString)

job_elements = results.find_all("div", class_="card-content")

for job_element in job_elements:
    title_element = job_element.find("h2", class_="title")
    company_element = job_element.find("h3", class_="company")
    location_element = job_element.find("p", class_="location")
    publication_date = job_element.find("time")
    link_url = job_element.find_all("a")[1]["href"]
    cadeaSQL = f'''INSERT INTO fakejob(position, company, address, pubDate, url) VALUES(
                    '{title_element.text.strip()}',
                    '{company_element.text.strip()}', 
                    '{location_element.text.strip()}', 
                    '{publication_date.text.strip()}', 
                    '{link_url}')'''
    result=engine.execute(cadeaSQL)


# Outros

In [ ]:
# Busca o contido da cabeceira H1
soup.h1.text

# Mostra a ruta á imaxe que se mostra na web
soup.img.get('src')

# Mostra o texto alternativo da imaxe
soup.img.get('alt')

# Mostra todos os textos "strongly"-resaltados da páxina
soup.find_all('strong')

# Mostra todos os enlaces presentes na páxina
for i in soup.find_all('a'):
    print(i.get('href'))

# Mostra os textos/palabras que teñen enlace 
# Mostra todos os enlaces presentes na páxina
for i in soup.find_all('a'):
    print(i.text)

# Conta o número de parágrafos presentes na páxina web
contador = 0
for i in soup.find_all('p'):
    contador = contador + 1
contador
# ou directamente utilizar len
# len(soup.find_all('p'))

# Mostra o contido do último parágrafo
soup.find_all('p')[-1].text

 

# NOVA PÁXINA: https://bigdatawirtz.github.io/exemplo-web/08.html
# Bótalle unha ollada ao código da páxina
url = 'https://bigdatawirtz.github.io/exemplo-web/08.html'
paxina = requests.get(url)

print(paxina.text)

# Parsear o contido da web
soup = BeautifulSoup(paxina.content, 'html.parser')

# Mostra o título da páxina
soup.title.text

# Mostra o charset da páxina, dentro de "meta"
soup.meta.get('charset')

# Conta o número de parágrafos que ten a páxina
len(soup.find_all('p'))

# Mostrar o texto no pé do artigo
soup.footer.p.text

# Mostrar o texto no pé da web
soup.find_all('footer')[-1].p.text

# Mostrar id da sección
soup.section.get('id')



**Fonte** (adaptado de): https://realpython.com/beautiful-soup-web-scraper-python/